In [2]:
import sys
from datetime import date
import pandas as pd
import numpy as np
from jugaad_data.nse import stock_df
import seaborn as sns
import ta
import matplotlib.pyplot as plt

In [3]:
def graph(macd_line, signal_line):
    plt.figure(figsize = (12, 6))
    sns.set_style('dark')
    sns.set_theme('paper')
    sns.lineplot(x = macd_line.index, y = macd_line, label = 'macd_line (SBIN)')
    sns.lineplot(x = signal_line.index, y = signal_line, label = 'signal_line (SBIN)')
    diff = signal_line-macd_line
    # sns.barplot(x = diff.index, y = diff, palette= ['red', 'green'], hue)
    plt.bar(diff.index, diff, color=['green' if val > 0 else 'red' for val in diff], width=1)

In [4]:
def linear(test_data, data):
    df = pd.DataFrame()
    df['P. CLOSE'] = data['PREV. CLOSE']
    df['P. OPEN'] = data['OPEN'].shift(1) 
    df['P. VWAP'] = data['VWAP'].shift(1)
    df['P. LOW'] = data['LOW'].shift(1) 
    df['P. HIGH'] = data['HIGH'].shift(1)
    df['P. NO OF TRADES'] = data['NO OF TRADES'].shift(1)
    df['OPEN'] = data['OPEN']
    df['ONES'] = 1
    df = df.drop(index=0)
    matrix = np.array(df)

    x_matrix = np.roll(matrix, shift=1, axis=1)
    y_matrix = data['CLOSE'].drop(index=0)
    
    x_transpose = x_matrix.transpose()
    x_transpose_x = np.array(x_transpose.dot(x_matrix))
    inverse = np.linalg.inv(x_transpose_x)
    x_transpose_y = np.array(x_transpose.dot(y_matrix))
    params = np.array(inverse.dot(x_transpose_y))
    
    print(params)

In [31]:
def pairs(df1, df2):
    spread = pd.DataFrame(df1['CLOSE'] - df2['CLOSE'])
    sq_spread = spread*spread
    roll = spread.rolling(window = 20).mean()
    sumOfSquares = sq_spread.rolling(window = 20).sum()
    
    variance = (sumOfSquares/20 - (roll*roll))
    sd = variance**0.5
    z_score = np.array((spread-roll)/sd)
    stock = 0
    cashflow = 0
    for i, score in enumerate(z_score):
        if score > 2 and stock > -5:
            stock -= 1
            cashflow += df1['CLOSE'][i]
            cashflow -= df2['CLOSE'][i]
        if score < -2 and stock < 5:
            stock += 1
            cashflow -= df1['CLOSE'][i]
            cashflow += df2['CLOSE'][i]
        print(cashflow)

In [32]:
# ------------------------------------ generation of data frame --------------------------------------------
def generate_dataframe(symbol, today, lastday, train_data):
    # formatting the date
    to_day = int(today[:2])
    to_month = int(today[3:5])
    to_year = int(today[6:10])
    
    la_day = int(lastday[:2])
    la_month = int(lastday[3:5])
    la_year = int(lastday[6:10])
    
    df = pd.DataFrame(stock_df(symbol=symbol, from_date=date(to_year, to_month, to_day), to_date=date(la_year, la_month, la_day), series="EQ"))
    df = df[[ "DATE", "CLOSE", "HIGH", "LOW", "PREV. CLOSE", "VWAP", "NO OF TRADES", "OPEN"]]
    df = df.iloc[::-1]
    df['DATE'] = pd.to_datetime(df['DATE'], format='%d-%m-%Y')
    df['DATE'] = df['DATE'].dt.strftime('%d/%m/%Y')
    if (train_data == "0"):
        df.to_csv("Stocks/"+symbol+".csv", index=False)
    else:
        df.to_csv("Stocks/"+symbol+"_train.csv", index = False)

# ----------------------------------------------- pickle --------------------------------------------------------
def write_pickle(DATA, symbol):
    pd.to_pickle(DATA, symbol + ".pkl")

# -------------------------------------------------------- MAIN -----------------------------------------------------------------------
def main():
    arguments = ["SBIN", "ADANIENT"]
    lastday = "01/01/2024"
    today = "05/12/2022"
    train_data = "0"
    for i, argument in enumerate(arguments):
        generate_dataframe(argument, today, lastday, train_data)
    #write_pickle(DATA, "Stocks/" + argument)
    
    df1 = pd.read_csv('Stocks/SBIN.csv')
    df2 = pd.read_csv('Stocks/ADANIENT.csv')
    pairs(df1, df2)

if __name__ == "__main__":
    main()

          DATE   CLOSE    HIGH     LOW  PREV. CLOSE    VWAP  NO OF TRADES  \
0   05/12/2022  617.30  618.00  607.55       607.55  614.05        210728   
1   06/12/2022  608.95  619.80  607.80       617.30  613.29        190699   
2   07/12/2022  607.05  612.90  604.50       608.95  607.96        113416   
3   08/12/2022  611.65  613.80  607.15       607.05  611.14        146277   
4   09/12/2022  616.50  618.00  609.10       611.65  613.84        169991   
5   12/12/2022  613.05  618.70  611.00       616.50  613.74        135955   
6   13/12/2022  616.75  617.40  612.50       613.05  615.45        162670   
7   14/12/2022  625.50  626.75  617.50       616.75  623.52        187174   
8   15/12/2022  615.95  629.55  614.30       625.50  621.92        148853   
9   16/12/2022  603.35  615.60  602.10       615.95  607.83        160875   
10  19/12/2022  604.45  609.50  603.00       603.35  605.78        149442   
11  20/12/2022  604.45  606.50  599.55       604.45  602.65        128701   